In [3]:
from typing import Dict, TypedDict, List
from langgraph.graph import StateGraph, START, END

In [4]:
class AgentState(TypedDict):
    number1 : int
    operation : str
    number2 : int
    finalnumber : int

In [8]:
def adder(state:AgentState) -> AgentState:
    """This is the node process add"""

    state["finalnumber"] = state["number1"] + state["number2"]

    return state

def subtractor(state:AgentState) -> AgentState:
    """This is the node process minus"""

    state["finalnumber"] = state["number1"] - state["number2"]

    return state

def decide_next_node(state:AgentState) -> AgentState:
    """This node will select the next node of the graph"""

    if state["operation"] == "+":
        return "additional_operation"
    elif state["operation"] == "-":
        return "substraction operation"

In [13]:
graph = StateGraph(AgentState)

graph.add_node("adder_node",adder)
graph.add_node("subtractor_node",subtractor)
graph.add_node("router_node",lambda state:state) #passthrough function, only do comparison instead of assignment

graph.add_edge(START, "router_node")
graph.add_conditional_edges(
    "router_node",
    decide_next_node,
    {
        "additional_operation": "adder_node",
        "subtraction_operation": "subtractor_node"
    }
)

graph.add_edge("adder_node", END)
graph.add_edge("subtractor_node", END)

app = graph.compile()


In [15]:
result = app.invoke({"number1":10,"number2":5,"operation":"+"})
result

{'number1': 10, 'operation': '+', 'number2': 5, 'finalnumber': 15}